This is a collection of all useful functions or Macros for data cleaning and modifications. This is for learning purposes.


Created by: Grace Choo

Date of Creation: 02 November 2022

Note: The ones with Header 2/Level 2 headings are explainations on how it should/can be used. Pay attention to the list of python packages needed to run the macro successfully.

# Export df to sql server

In [ ]:
import urllib
from sqlalchemy import create_engine

def to_sql(df, table, server, database):
    odbc_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;'
    connect_str = 'mssql+pyodbc:///?odbc_connect=' + urllib.parse.quote_plus(odbc_str)
    engine = create_engine(connect_str)
    df.to_sql(table, engine, if_exists = "replace", index=False, chunksize = 100)

## To Execute

In [ ]:
to_sql(df, 'Insert your sql table name here', 'Insert your server name here', 'Insert your database here')

# Encrypt and Decrypt Functions

In [ ]:
def get_cipherletter(new_key, letter):
    #still need alpha to find letters
    alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    
    if letter in alpha:
        return alpha[new_key]
    else:
        return letter

def encrypt(key, message):
    message = message.upper()
    alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    result = ""

    for letter in message:
        new_key = (alpha.find(letter) + key) % len(alpha)
        result = result + get_cipherletter(new_key, letter)
    return result

def decrypt(key, message):
    message = message.upper()
    alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    result = ""

    for letter in message:
        new_key = (alpha.find(letter) - key) % len(alpha)
        result = result + get_cipherletter(new_key, letter)
    return result

## Test to encrypt column

In [ ]:
df['Unique_ID1'] = df['Unique_ID'].apply(lambda x: encrypt(-8, x))

## Test to decrypt column

In [ ]:
df['Unique_ID2'] = df['Unique_ID1'].apply(lambda x: decrypt(-8, x))

# Last Day of the Month

In [ ]:
import datetime as dt
from dateutil.relativedelta import relativedelta
def last_day_of_month(datevar):
    #if it is december, then straight away assign to 31 DEC YYYY
    if datevar.month == 12:
        return datevar.replace(day=31)
    #if it is other months then first force the date to be 01 MMM YYYY and then move the date to 1 month later then -1 day.
    return dt.date(datevar.year, datevar.month, 1) + relativedelta(months=1) -  relativedelta(days=1)

## Test the macro

In [ ]:
import datetime as dt
from datetime import date
from dateutil.relativedelta import relativedelta

#Today's Date
today = date.today()
print("Today's date is: ",today)

#Use last_day_of_month(datevar) macro
print("Last Day of the month is : ", last_day_of_month(today))

# Timer Start and Ends

In [ ]:
import datetime as dt
import timeit

def Macro_CountTimerStarts():
    currentDT = dt.datetime.now()
    print("Program Starts Now:")
    print (currentDT.strftime("%I:%M:%S %p"))

#Record Date time
time_start = timeit.default_timer()


def Macro_CountTimerEnds():
    #End Program
    time_taken = timeit.default_timer() - time_start
    minutes_taken = str(dt.timedelta(seconds=time_taken))
    
    EndDT = dt.datetime.now()
    print("Program Ends Now:")
    print (EndDT.strftime("%I:%M:%S %p"))
    print(f"Time taken: {minutes_taken}")


## Test the macro

In [ ]:
Macro_CountTimerStarts()

In [ ]:
Macro_CountTimerEnds()

# List out file names using patterns

In [ ]:
import os, fnmatch
import os.path


def F_findFile(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(name))
    return result


## To execute

In [ ]:
#To find file with names contains 'GIGT' and .xlsx format
F_findFile('*GIGT*.xlsx', r'C:\Users\Grace.Choo\OneDrive - Zurich APAC')

# Keep numbers only

In [ ]:
#To only keep numbers in pandas column. like for cleaning of NewIC numbers:
import re as re
def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)

## To execute

In [ ]:
df['COL1']=df['COL1'].apply(lambda x: find_number(x))

# Identify Commercial Customers based on their Name

In [ ]:
def ClientTypeC(df,LookupNameCol,ClientTypeCol):
    
    df[LookupNameCol]=df[LookupNameCol].str.upper()
    df[LookupNameCol]=df[LookupNameCol].str.replace('\s+', ' ', regex=True)
   
    
    df[LookupNameCol]=df[LookupNameCol].astype(str).str.replace("SDNBHD","SDN BHD")
    df[LookupNameCol]=df[LookupNameCol].astype(str).str.replace("SDNBIN","SDN BHD")
    
    df.loc[df[LookupNameCol].str.contains(r'\bPARTNER\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bINTERNATIO\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bFINANC\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\b ADVIS\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\b CAPIT\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bWEALTH \b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bSUNGAI PETANI\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bDIRECT ACCOUNTS\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bINSTITUTIONAL CLIENTS\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPOS MALAYSIA\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bMBSB\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bDINERS CLUB\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPLAZA ZURICHPJ\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\b DIRECT\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\b CONSUL\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\b BROKER\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bEMPIRE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bADVISORY\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bAGENCIES\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bAGENCY\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bAGENT\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bBANK\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bGROUP\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCONSULTING\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPOS TAKAFUL\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bWEALTH PLANNERS\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bAUTO \b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bVENTURE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bAUTOTECH\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCONSULTANT\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bURUSNIAGA\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bSDN BHD\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bASSOCIATES\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bDAGANG\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\b CAFE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bNETWORKS\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bHEALTHCARE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bAGENCY\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bSDN \b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bBHD\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bSENDIRIAN\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bBERHAD\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCONSTRUCT\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bTRADE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bTRADING\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bENTERP\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bENGINEE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bINDUS\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bDEVELOP\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bMAJL\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bKEDAI\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bHOLDI\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bUNIVE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bSERVI\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bSYARIKAT\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCORPOR\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCENTRE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bMANUFAC\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bACADEMY\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bAUTO PART\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bSOLUTION\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bWORLD\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCOLLECTIONS\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bKLINIK\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCHILDCARE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bTABUNG\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bWISMA \b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bTECHNIC\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPUSAT\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPOLIKLINIK\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPERNIAGAAN\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPERBADANAN\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPELANDUK EMAS\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bTELESHOP\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCONSULTANCY\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bOPTIKAL\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCOLLECTION\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bRESOURCES\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bFARMASI\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bHARDWARE\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bAPOTEK\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bEDUCATION\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bMARKETING\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bMANAGEMENT\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bEQUIPMENT\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPRINTING SDN BHD\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPRINTING\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCOMMUNICATION\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bACCESSORY\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bTELECOMMUNICATION\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bKERAJAAN MALAYSIA\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bPEMBANGUNAN\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bCONTRACTOR\b',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'\bELECTRICAL\b',regex=True),ClientTypeCol]='C'
    
    #if Ends with:
    df.loc[df[LookupNameCol].str.contains(r'BHD$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'BROKERS$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'CAPITAL$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'FINANCIAL$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'FREEDOM$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'PLANNERS$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'PLANNING$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'RINGGITPLUS$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'PARTNERS$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'LEGACY$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'SYSTEM$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'DISTRIBUTION$',regex=True),ClientTypeCol]='C'
    df.loc[df[LookupNameCol].str.contains(r'ADVISORS$',regex=True),ClientTypeCol]='C'
    
    df.loc[df[ClientTypeCol].str.contains(r'\bC\b',regex=True) &
           (df[LookupNameCol].str.contains(r'\bA/P\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bA/L\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bS/O\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bD/O\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bSINGH\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bAP\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bAL\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bBIN\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bSO\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bDO\b',regex=True) |
            df[LookupNameCol].str.contains(r'\bBINTI\b',regex=True)
            )
           ,ClientTypeCol]='' 
    
    return df
    

## To execute

In [ ]:
df = ClientTypeC(df,'{the name of your column that contains names}',
                 '{the name of the column that provides the commercial column indicator}')

(Not used or tested yet)Convert multiple datetime columns into date columns

In [ ]:
def ConvertDateTimeIntoDate(df, DateTimeCol):
    for col in DateTimeCol:
        TempCol = 'T_' + col
        df.rename(columns = {col: TempCol}, inplace = True)
        df[col] = pd.DatetimeIndex(df[TempCol]).date
        df.drop([TempCol], axis = 1, inplace = True)

ConvertDateTimeIntoDate(GI_AgentList, DateTimeCol)

 To Execute

In [1]:
DateTimeCol = ['Col1','Col2','Col3']
ConvertDateTimeIntoDate(YourdfName, DateTimeCol)

# To Import all macros

Note: This should produce a message 'Import Common Macros Complete!' if it is executed correctly.

In [ ]:
exec(open(r'C:\Users\Grace.Choo\OneDrive - Zurich APAC\Projects\Python Project\202210 Convert SAS Code to Python Code Azure Migration\Common Macros.py').read())